In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from sklearn.metrics import r2_score
import os
from shapely.geometry import shape
import csv
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import fiona
import rasterio
import rasterio.transform
import rasterio.mask
from fiona import Feature, Geometry
from shapely.geometry import mapping, shape

In [47]:
#extracting the datasets

#CALVING FLUX. I take the ones generated by the tif files in the folder 'exploration_ice_calving'
data_dir_ice_calving = '/Users/francesco/Desktop/Thesis/exploration_ice_calving/'
calving_df= pd.read_csv( data_dir_ice_calving+ 'Glacier_ice_calving_withice_shelves_flux_greene.csv', index_col = 0)
calving_df = calving_df.sort_index()
index_calving = calving_df.index.values
calving_columns= calving_df.columns
#create an array with the common years, from 2000 to 2017
common_years = np.arange(2000,2018)
calving = calving_df[common_years.astype(str)]


#BASAL MELTING, retrived from the csv file 'Merged_Integrated_melt_rates.csv'. the index is NOT the same as the calving one
datadir ='/Users/francesco/Desktop/Thesis/Data/'
df_melt_rates = pd.read_csv(datadir + 'Merged_Integrated_melt_rates.csv')
df_err_melt_rates = pd.read_csv(datadir + 'Merged_Integrated_melt_err_rates_second_method.csv')


#Importing the RIGNOT dataset
df_rignot = pd.read_excel('table.xlsx')
df_rignot = df_rignot.set_index('id')
df_rignot = df_rignot.sort_index()

#NB: THE RIGNOT INDEX SHOULD HAVE THE CORRECT VALUES OF THE DF INDEX, BEACUSE I WROTE IT MANUALLY ON THE TABLE EXCEL FILE
df_rignot_index = df_rignot.index.values


/Users/francesco/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [48]:
#Extracting the info from the shapefile 'ice_shelf.shp'

shapefile_path = '/Users/francesco/Desktop//Thesis/Data/ice_shelf.shp'

ids = []
Names = []
lat_lon_coords = []
regions = []
areas = []

shapefile=fiona.open(shapefile_path)

print(shapefile.schema['properties'])

for feature in shapefile:

    id = feature['properties']['id']
    name=feature['properties']['name']
    geometry=feature['geometry']
    if geometry is None:
        continue
    region = feature['properties']['regions']
    lat = feature['properties']['latitude']
    lon = feature['properties']['longitude']
    area = feature['properties']['area']

    bounds=shape(geometry).bounds

    #Taking the info
    ids.append(id)
    Names.append(name)
    lat_lon_coords.append([lat,lon])
    regions.append(region)
    areas.append(area)


{'id': 'int:10', 'name': 'str:30', 'regions': 'str:11', 'type': 'str:2', 'is_index': 'int:10', 'ice_shelf_': 'str:50', 'measures_n': 'str:50', 'latitude': 'float:24.15', 'longitude': 'float:24.15', 'thickness_': 'float:24.15', 'thicknes_1': 'float:24.15', 'smb_mean_9': 'float:24.15', 'smb_mean_1': 'float:24.15', 'ss_gl_flux': 'float:24.15', 'ss_gl_fl_1': 'float:24.15', 'ss_calving': 'float:24.15', 'ss_calvi_1': 'float:24.15', 'model_gl_f': 'float:24.15', 'instant_co': 'float:24.15', 'mass_loss_': 'float:24.15', 'mass_los_1': 'float:24.15', 'mass_los_2': 'float:24.15', 'mass_los_3': 'float:24.15', 'mass_los_4': 'float:24.15', 'mass_los_5': 'float:24.15', 'mass_los_6': 'float:24.15', 'mass_los_7': 'float:24.15', 'mass_los_8': 'float:24.15', 'mass_los_9': 'float:24.15', 'mass_los10': 'float:24.15', 'mass_los11': 'float:24.15', 'mass_los12': 'float:24.15', 'mass_los13': 'float:24.15', 'mass_los14': 'float:24.15', 'mass_los15': 'float:24.15', 'mass_los16': 'float:24.15', 'mass_los17': 'floa

In [49]:
#Create a dataset with the iformatinos found above
df = pd.DataFrame({ 'Names':Names, 'lat_lon_coords':lat_lon_coords, 'regions':regions, 'area':areas}, index = ids)
df

,Names,lat_lon_coords,regions,area
1,Dawson_Lambton,"[-76.11779165, -26.75288573]",East,2.425658e+07
2,Philbin_Inlet,"[-74.02459031, -114.0719424]",West,8.962356e+07
3,Walgreen_Coast_2,"[-73.69275898, -101.6811588]",West,1.406183e+07
4,Jackson,"[-74.71420356, -135.7763549]",West,5.952413e+07
5,LarsenD_1,"[-69.5601021, -61.80408688]",Peninsula,3.901872e+07
...,...,...,...,...
181,Rydberg_Peninsula_2,"[-73.27738519, -79.08368328]",Peninsula,3.109665e+06
162,Ninnis,"[-68.24100008, 147.4658576]",East,1.929034e+09
163,Suvorov,"[-69.92280596, 160.3356633]",East,2.136963e+08
164,Walgreen_Coast_1,"[-73.96258239, -101.7579489]",West,8.783072e+07


### Calving Comparison Rignot-Me for 2008

In [50]:
#Trying 2008
calving_2008 = calving['2008']

In [51]:
#Selecting the common index between calving and Rignot
common_index_calving_rignot = np.intersect1d(index_calving, df_rignot_index) #indices in common between calving and rignot

# Creating the sub-dataset of rignot and calving which contain just the common indices
df_rignot_common = df_rignot.loc[common_index_calving_rignot]
calving_2008_common = calving_2008.loc[common_index_calving_rignot]


In [52]:
#Create a dataset comparing Calving Rignot-Me

comparison_calving_rignot_me = pd.DataFrame({'name':df_rignot_common.name,'calving_by_me':calving_2008_common, 'calving_rignot':df_rignot['Ice-front flux'], 'percentage': df_rignot['Ice-front flux']/calving_2008_common }, index=common_index_calving_rignot)
comparison_calving_rignot_me
comparison_calving_rignot_me.to_csv('comparison_rignot_calving.csv')

In [53]:
tot_calving_me = calving_2008_common.sum()
tot_calving_rignot = df_rignot_common['Ice-front flux'].sum()

print('Total calving by me: ' , tot_calving_me , ' total calving by rignot: ', tot_calving_rignot)


Total calving by me:  1475.8420447635244  total calving by rignot:  1081.3


### Melting Comparison Rignot-Me

In [54]:
# The melt index is not the correct one. TRYING 2008 NOW    

df_melt_rates
df_melt_rates_2008 = df_melt_rates.loc[:,['Names', '2008']]
df_melt_rates_2008

,Names,2008
0,Dawson_Lambton,-0.08
1,Philbin_Inlet,4.95
2,Walgreen_Coast_2,0.04
3,Jackson,-1.26
4,LarsenD_1,-0.01
...,...,...
178,Rydberg_Peninsula_2,0.00
179,Ninnis,-9.21
180,Suvorov,0.08
181,Walgreen_Coast_1,0.16


In [55]:
#I now select the galciers which are in common between rignot and melt rates, according to the names

common_names_melt_rignot = np.intersect1d(df_melt_rates_2008.Names, df_rignot_common.name)
print(np.shape(common_names_melt_rignot))

#Selecting the common glaciers from df_melt
df_melt_common = df_melt_rates_2008.loc[df_melt_rates.Names.isin(common_names_melt_rignot)]
df_melt_common #Till here the values are correct


(61,)


,Names,2008
5,LarsenD,9.70
7,LarsenC,81.18
9,WilmaRobertDowner,0.60
10,Shirase,6.72
12,Rayner_Thyer,2.71
...,...,...
167,Holmes,-3.46
168,Moscow_University,28.06
169,Prince_Harald,-7.35
175,LarsenE,6.12


In [56]:
#Rebuilding the index for the d_f_melt_rates_common
df_melt_index = []
df_melt_names_common_temp = []
df_melt_melt_temp = []
df_melt_rignot_temp = []
df_melt_rignot_temp_Gton = []



#In this cycle i check when the name in the 2 dataset are the same, in which case I save the name,  the melt by Rignot and by me and the index in Rignot

j=0
for name_rignot in df_rignot_common.name:
    i =0
    for name_melt in df_melt_common.Names:

        if name_melt == name_rignot:
            df_melt_index.append(df_rignot_common.index[j]) 
            df_melt_names_common_temp.append(name_melt)
            df_melt_melt_temp.append(df_melt_common['2008'].iloc[i])
            df_melt_rignot_temp.append(df_rignot_common['Basal melting per meter m/yr'].iloc[j])
            df_melt_rignot_temp_Gton.append(df_rignot_common['Basal melting tot Gt/yr'].iloc[j])
            
        i = i+1

    j = j+1

In [57]:
melt_comparison_rignot_me = pd.DataFrame({'name':df_melt_names_common_temp, 'melt_by_me':df_melt_melt_temp, 'melt_by_rignot_Gt_yr': df_melt_rignot_temp_Gton, 'ratio_rignot_me': np.array(df_melt_rignot_temp_Gton)/np.array(df_melt_melt_temp)}, index=df_melt_index)
#i want to sort the dataframe by index
melt_comparison_rignot_me = melt_comparison_rignot_me.sort_index()
melt_comparison_rignot_me
melt_comparison_rignot_me.to_csv('comparison_rignot_melting.csv')


In [58]:
tot_melt_me = np.array(df_melt_melt_temp).sum()
tot_melt_rignot = np.array(df_melt_rignot_temp_Gton).sum()

print('Total melt by me: ' , tot_melt_me , ' total melt by rignot: ', tot_melt_rignot)

Total melt by me:  1554.1200000000001  total melt by rignot:  1310.2000000000003


### I now want to do a comparison between the calving flux and the melt flux

In [59]:
#Here I go through the calving dataset and I add the names where there is the correct index
for index_value in calving.index:

    name = df.loc[index_value, 'Names']
    calving.loc[index_value, 'Names'] = name

calving

/var/folders/9n/18cd5sld4lv1jvk0nw84p_bh0000gq/T/ipykernel_29515/3768571545.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calving.loc[index_value, 'Names'] = name


,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,Names
1,0.999649,0.313843,0.790420,0.732301,0.000000,0.836916,0.499825,0.232477,0.848539,0.000000,0.000000,0.557944,0.337091,0.418458,0.464953,0.000000,0.220853,0.069743,Dawson_Lambton
2,0.370000,0.000000,0.290000,0.210000,0.000000,0.320000,0.000000,0.020000,1.170000,0.000000,0.310000,0.000000,0.050000,0.110000,0.180000,0.000000,0.000000,0.000000,Philbin_Inlet
3,0.000000,0.000000,0.149756,0.138236,0.000000,0.138236,0.161276,0.149756,0.011520,0.000000,0.138236,0.000000,0.000000,0.138236,0.126717,0.000000,0.000000,0.000000,Walgreen_Coast_2
4,2.910863,0.286153,0.868325,0.740050,0.000000,0.000000,0.029602,0.059204,1.430763,0.000000,0.000000,0.069071,0.374959,0.601907,0.256551,0.000000,0.000000,0.000000,Jackson
5,0.285593,0.118176,0.147720,0.009848,0.000000,0.206808,0.147720,0.167416,0.295441,0.118176,0.196960,0.118176,0.177264,0.177264,0.216656,0.226504,0.226504,0.216656,LarsenD_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,0.260000,0.000000,0.140000,0.040000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.040000,0.040000,Britten
177,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ClarkeBay
178,0.000000,0.000000,0.372028,0.420979,0.283916,0.000000,0.362238,0.156643,0.000000,0.000000,0.303496,0.558042,0.117482,0.450349,0.009790,0.000000,0.048951,0.029371,Hayes_Coats_Coast
180,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,Rydberg_Peninsula_1


In [60]:
#Basal Melting USED, retrived from the csv file 'Merged_Integrated_melt_rates.csv'. 

#I had changed manually the index in the excel file, looking in the iceshelf_flux_greene.gpkg file, checking the attibute table

datadir ='/Users/francesco/Desktop/Thesis/Data/'
melt_rates = pd.read_excel(datadir + '/Merged_Integrated_melt_rates_with_id_manually_added.xlsx', index_col=0)
melt_rates.sort_index(inplace=True)

In [61]:
#Check if there are repetead values in the index of the dataset df_melt_rates
melt_rates.index.is_unique

#What are the repeated numbers?
unique, counts = np.unique(melt_rates.index, return_counts=True)

# print the repeated values
print(unique[counts > 1])

[nan]


In [62]:
#Working on the compatibility of the 2 datasets, calving from tif and melt rates from the csv file

#Selecting the common index, and converting the datasets
common_index_calving_melting = np.intersect1d(calving.index, melt_rates.index)

melt_rates_common = melt_rates.loc[common_index_calving_melting]
calving_common = calving.loc[common_index_calving_melting]

#Moving the names to the first column in the calving dataset
names_temp = calving_common.pop('Names')
calving_common.insert(0, 'Names', names_temp)

#Finding the common columns, and converting them to strings (in order to do the comparison)
cols1 = np.array(melt_rates_common.columns) #columns of the melt rates
cols2 = np.array(calving_common.columns) #columns of the calving dataset
melt_rates_common.columns.astype(str)
common_cols = np.intersect1d(cols1.astype(str), cols2)

#Convert the columns to string
melt_rates_common.columns = melt_rates_common.columns.astype(str)

#creating the subset of melt_rates, with common columns
melt_rates_common = melt_rates_common.loc[:,common_cols]
names_temp = melt_rates_common.pop('Names')
melt_rates_common.insert(0, 'Names', names_temp)


In [63]:
years = np.arange(2000, 2018, 1)
years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [64]:
ids = []
Names = []
melt = []
calving = []
tot =[]

Years = np.arange(2000, 2018, 1)

for index_value in melt_rates_common.index:
    ids.append(index_value)
    Names.append(melt_rates_common.loc[index_value, 'Names'])

    #For melt rates common, caving and total i add the whole time serie
    melt.append(melt_rates_common.loc[index_value, Years.astype(str)].values)
    calving.append(calving_common.loc[index_value, Years.astype(str)].values)
    tot.append(melt_rates_common.loc[index_value, Years.astype(str)].values + calving_common.loc[index_value, Years.astype(str)].values)

         

In [65]:
#Dataset with ids and time_series

df_comparison_melting_calving_final = pd.DataFrame({'Names':Names, 'melt_time_serie':melt, 'calving_time_serie':calving, 'tot':tot}, index=ids)


In [66]:
from matplotlib.backends.backend_pdf import PdfPages

''' 

with PdfPages('Glaciers calving and melting comparison.pdf') as pdf:

    for id in df_comparison_melting_calving_final.index:

        fig = plt.figure()

        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['melt_time_serie'],label =  ' Melting rate ')
        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['calving_time_serie'],label = ' Calving')
        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['tot'],label = ' tot')
        

        plt.legend()
        plt.xlabel('Years')
        plt.ylabel('Ice Loss (Gt/yr)')

        #In the title I want to print the R2 value, rounded with 2 decimals
        plt.title('Melting rates of galcier ' + str(df_comparison_melting_calving_final.loc[id]['Names']) + ', its id is: ' + str(id) )




        pdf.savefig(fig) 
        plt.close()
'''

" \n\nwith PdfPages('Glaciers calving and melting comparison.pdf') as pdf:\n\n    for id in df_comparison_melting_calving_final.index:\n\n        fig = plt.figure()\n\n        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['melt_time_serie'],label =  ' Melting rate ')\n        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['calving_time_serie'],label = ' Calving')\n        plt.plot(Years, df_comparison_melting_calving_final.loc[id]['tot'],label = ' tot')\n        \n\n        plt.legend()\n        plt.xlabel('Years')\n        plt.ylabel('Ice Loss (Gt/yr)')\n\n        #In the title I want to print the R2 value, rounded with 2 decimals\n        plt.title('Melting rates of galcier ' + str(df_comparison_melting_calving_final.loc[id]['Names']) + ', its id is: ' + str(id) )\n\n\n\n\n        pdf.savefig(fig) \n        plt.close()\n"

In [67]:
#Problems with Wordie_(Harriott) AND Wordie_(Harriott_Headland). But the correction I had done is right, the problem is probably on claving dataset

#Check if they are unique
print(melt_rates_common.index.is_unique)
print(calving_common.index.is_unique)

#Check if they have the same index
if np.array_equal(melt_rates_common['Names'], calving_common['Names']):
    print('The two dataframes have the same index')
else:
    print('The two dataframes have different index')

#spot which is the element which is different
for i in range(len(melt_rates_common['Names'])):
    if melt_rates_common['Names'].iloc[i] != calving_common['Names'].iloc[i]:
        print('The element which is different is: ', melt_rates_common['Names'].iloc[i], ' and it is in position: ', i)



True
True
The two dataframes have different index
The element which is different is:  Wordie_(Harriott)  and it is in position:  42
The element which is different is:  Wordie_(Harriott_Headland)  and it is in position:  43


In [68]:
### Check the results with GREEN (calving) AND PAOLO (melting)

### Check the results with Green paper

In [172]:
#CALVING FLUX. I take the ones generated by the shp files in the folder 'exploration_ice_calving'
#IN this shp files the negative values were put to 0
data_dir_ice_calving = '/Users/francesco/Desktop/Thesis/exploration_ice_calving/'
calving_df= pd.read_csv( data_dir_ice_calving+ 'df_ice_calving_shp_negative_and_positive.csv', index_col = 0)
calving_df = calving_df.sort_index()
index_calving = calving_df.index.values

#calving_df.insert(0, 'Names', df['Names'])
calving_df

#Cumulative sum of the calving fluxes, (excluding the first column, which contains the Names)
calving_df_cumsum = calving_df.iloc[:,1:].cumsum(axis=1)
calving_df_cumsum.insert(0, 'Names', df['Names'])



In [173]:
#Calculating the total loss from the calving fluxes from the shp files

calving_df_cumsum
calving_df_cumsum_2021 = calving_df_cumsum.iloc[:,-1]
tot = calving_df_cumsum_2021.sum()
tot

-1807.4199999999998

In [174]:
ids_roi = [56,8,34,70,35]

In [175]:
calving_df_cumsum_roi = calving_df_cumsum.loc[ids_roi]
calving_df_cumsum_roi

,Names,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
id_measures,,,,,,,,,,,,,,,,,,,,,,,
56,Thwaites,0.39,41.49,-77.03,-76.44,-197.43,-130.05,-78.13,-34.59,18.15,37.78,-215.41,-252.85,-728.67,-1531.24,-1600.82,-1617.14,-1682.13,-1723.41,-1770.77,-1777.63,-1774.80,-1897.70
8,LarsenC,69.91,48.68,75.75,94.49,89.30,-204.53,-188.58,-154.78,-105.04,-103.72,-100.81,-69.50,-34.89,8.98,14.25,46.86,75.63,109.13,-1176.29,-1169.04,-1180.25,-1164.71
34,Ronne,-1269.23,-1283.00,-1131.97,-978.22,-853.72,-705.59,-606.82,-412.66,-246.58,-192.58,-27.06,135.91,298.32,452.12,577.12,711.56,825.17,955.17,1083.37,1216.59,1357.06,1479.73
70,Ross_West,-2839.96,-2800.02,-2682.68,-2573.69,-2570.47,-2491.13,-2391.75,-2322.52,-2133.36,-2095.00,-2011.58,-1884.31,-1797.68,-1680.79,-1589.05,-1484.50,-1389.10,-1271.55,-1174.73,-1061.29,-953.43,-871.95
35,Filchner,207.74,160.47,244.91,328.24,372.69,428.42,513.85,597.23,710.64,734.68,810.17,903.47,982.42,1071.33,1131.56,1227.38,1311.77,1400.90,1491.93,1577.56,1672.84,1758.29


In [159]:
calving_df_cumsum['tot'] = calving_df_cumsum.sum(axis=1)

/var/folders/9n/18cd5sld4lv1jvk0nw84p_bh0000gq/T/ipykernel_29515/3158215211.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  calving_df_cumsum['tot'] = calving_df_cumsum.sum(axis=1)


In [160]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
calving_df_cumsum.loc[56]

Names    Thwaites
2000         0.39
2001        41.49
2002       -77.03
2003       -76.44
2004      -197.43
2005      -130.05
2006       -78.13
2007       -34.59
2008        18.15
2009        37.78
2010      -215.41
2011      -252.85
2012      -728.67
2013     -1531.24
2014     -1600.82
2015     -1617.14
2016     -1682.13
2017     -1723.41
2018     -1770.77
2019     -1777.63
2020      -1774.8
2021      -1897.7
tot     -17068.43
Name: 56, dtype: object

###Trying with the tif dataset

In [171]:
#CALVING FLUX. I take the ones generated by the tif files in the folder 'exploration_ice_calving'
data_dir_ice_calving_tif = '/Users/francesco/Desktop/Thesis/exploration_ice_calving/'
calving_df_tif= pd.read_csv( data_dir_ice_calving_tif+ '/Glacier_ice_calving_with_ice_shelves_flux_greene.csv', index_col = 0)
calving_df_tif = calving_df.sort_index()
index_calving_tif = calving_df_tif.index.values

#calving_df.insert(0, 'Names', df['Names'])
calving_df_tif


#I want to calculate the cumulative sum of the calving fluxes
#calving_df_cumsum = calving_df.cumsum(axis=1)

#print(calving_df)
#calving_df_cumsum = calving_df.cumsum(axis=1)
#Do the cumulative sum of the calving fluxes, excluding the first column Name
#calving_df_tif_cumsum = calving_df_tif.iloc[:,1:].cumsum(axis=1)
#calving_df_tif_cumsum.insert(0, 'Names', df['Names'])



,Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
id_measures,,,,,,,,,,,,,,,,,,,,,,,
1,Dawson_Lambton,0.86,0.27,0.68,0.63,-0.07,0.72,0.43,0.20,0.73,-0.36,-0.05,0.48,0.29,0.36,0.40,-0.74,0.19,0.06,0.13,0.06,0.05,-0.03
2,Philbin_Inlet,0.37,-0.34,0.29,0.21,-0.66,0.32,-0.61,0.02,1.17,-0.89,0.31,-0.61,0.05,0.11,0.18,-0.35,0.00,0.00,0.00,0.00,0.01,0.00
3,Walgreen_Coast_2,-0.63,-0.41,0.13,0.12,-0.35,0.12,0.14,0.13,0.01,-0.40,0.12,-0.30,-0.06,0.12,0.11,-0.43,0.00,0.00,0.00,0.02,-0.02,0.00
4,Jackson,2.95,0.29,0.88,0.75,-0.32,-0.69,0.03,0.06,1.45,-0.73,-1.38,0.07,0.38,0.61,0.26,-1.40,-0.10,0.00,0.00,0.03,0.03,0.00
5,LarsenD_1,0.29,0.12,0.15,0.01,-0.28,0.21,0.15,0.17,0.30,0.12,0.20,0.12,0.18,0.18,0.22,0.23,0.23,0.22,0.22,0.22,0.22,-2.03
6,LarsenD,10.52,-0.42,5.22,4.86,-0.99,4.80,3.76,3.91,5.43,1.85,4.36,3.83,2.97,3.44,-0.66,6.26,9.04,5.92,5.80,5.52,5.28,-97.03
7,LarsenF,-13.05,0.68,1.14,1.09,0.17,1.09,0.91,0.93,1.19,0.64,0.84,0.66,0.64,0.63,-1.07,-0.99,0.43,0.32,0.33,0.16,0.04,0.03
8,LarsenC,69.91,-21.23,27.07,18.74,-5.19,-293.83,15.95,33.80,49.74,1.32,2.91,31.31,34.61,43.87,5.27,32.61,28.77,33.50,-1285.42,7.25,-11.21,15.54
9,Telen,-0.33,-3.25,0.10,0.02,-0.01,-0.01,0.04,-0.02,-0.02,0.03,-0.02,0.04,-0.04,-0.06,0.00,-0.05,-0.02,0.01,0.01,0.05,-0.41,-0.09


In [169]:
calving_df_tif

,Names,Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
id_measures,,,,,,,,,,,,,,,,,,,,,,,,
1,Dawson_Lambton,Dawson_Lambton,0.86,0.27,0.68,0.63,-0.07,0.72,0.43,0.20,0.73,-0.36,-0.05,0.48,0.29,0.36,0.40,-0.74,0.19,0.06,0.13,0.06,0.05,-0.03
2,Philbin_Inlet,Philbin_Inlet,0.37,-0.34,0.29,0.21,-0.66,0.32,-0.61,0.02,1.17,-0.89,0.31,-0.61,0.05,0.11,0.18,-0.35,0.00,0.00,0.00,0.00,0.01,0.00
3,Walgreen_Coast_2,Walgreen_Coast_2,-0.63,-0.41,0.13,0.12,-0.35,0.12,0.14,0.13,0.01,-0.40,0.12,-0.30,-0.06,0.12,0.11,-0.43,0.00,0.00,0.00,0.02,-0.02,0.00
4,Jackson,Jackson,2.95,0.29,0.88,0.75,-0.32,-0.69,0.03,0.06,1.45,-0.73,-1.38,0.07,0.38,0.61,0.26,-1.40,-0.10,0.00,0.00,0.03,0.03,0.00
5,LarsenD_1,LarsenD_1,0.29,0.12,0.15,0.01,-0.28,0.21,0.15,0.17,0.30,0.12,0.20,0.12,0.18,0.18,0.22,0.23,0.23,0.22,0.22,0.22,0.22,-2.03
6,LarsenD,LarsenD,10.52,-0.42,5.22,4.86,-0.99,4.80,3.76,3.91,5.43,1.85,4.36,3.83,2.97,3.44,-0.66,6.26,9.04,5.92,5.80,5.52,5.28,-97.03
7,LarsenF,LarsenF,-13.05,0.68,1.14,1.09,0.17,1.09,0.91,0.93,1.19,0.64,0.84,0.66,0.64,0.63,-1.07,-0.99,0.43,0.32,0.33,0.16,0.04,0.03
8,LarsenC,LarsenC,69.91,-21.23,27.07,18.74,-5.19,-293.83,15.95,33.80,49.74,1.32,2.91,31.31,34.61,43.87,5.27,32.61,28.77,33.50,-1285.42,7.25,-11.21,15.54
9,Telen,Telen,-0.33,-3.25,0.10,0.02,-0.01,-0.01,0.04,-0.02,-0.02,0.03,-0.02,0.04,-0.04,-0.06,0.00,-0.05,-0.02,0.01,0.01,0.05,-0.41,-0.09
